In [30]:
# Importing needed packages
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [47]:
# Loading first page content
first_url = 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_acceptedeventrelation.htm'
emplacement_du_driver="/Users/macbook16decamille/chromedriver"
driver = webdriver.Chrome(emplacement_du_driver)
driver.get(first_url)

# Accepting cookies and closing footer
time.sleep(4)
driver.find_element_by_css_selector("button#onetrust-accept-btn-handler").click()
time.sleep(2)
driver.find_element_by_css_selector("img.close-icon").click()

# Getting the list of all the pages of the current version
liste_of_url_key=[]
for el in driver.find_elements_by_css_selector("span.ng-binding.ng-scope") :
    liste_of_url_key.append(el.text)

# Creating the list of all the url to scrape
idx=liste_of_url_key.index("AcceptedEventRelation")
idx2=liste_of_url_key.index("WorkTypeGroupMember")
liste_of_url_key=liste_of_url_key[idx:idx2]
for idx,url in enumerate(liste_of_url_key) : 
    liste_of_url_key[idx]=url.lower()

# Cleaning liste_of_url_key
remove_objects_list = ['article type__datacategoryselection','community (zone)','consumption rate','consumption schedule','eventrelation','listemailindividualrecipient','listemailrecipientsource','recordvisibility (pilot)','sessionpermsetactivation','salesforce surveys object model','topiclocalization—beta','transactionsecuritypolicy','usermembershipsharingrule','userrole','wishlist (beta)','wishlistitem (beta)']
else_objects_list = ['articletype__datacategoryselection','community','consumption_rate','consumption_schedule','eventattendee','listemail_individual_recipient','listemail_recipient_source','recordvisibility','sessionpermsetactivations','topiclocalization','transactionsecurity','usermembershiprule','role','wishlist','wishlistitem']
for el in remove_objects_list :
    if el in liste_of_url_key:
        liste_of_url_key.remove(el)
for el in else_objects_list :
    liste_of_url_key.append(el)
    
print(liste_of_url_key)

['acceptedeventrelation', 'account', 'accountbrand', 'accountcontactrelation', 'accountcleaninfo', 'accountcontactrole', 'accountinsight', 'accountownersharingrule', 'accountpartner', 'accountrelationship', 'accountrelationshipsharerule', 'accountshare', 'accounttag', 'accountteammember', 'accountterritoryassignmentrule', 'accountterritoryassignmentruleitem', 'accountterritorysharingrule', 'accountuserterritory2view', 'actioncadence', 'actioncadencerule', 'actioncadencerulecondition', 'actioncadencestep', 'actioncadencesteptracker', 'actioncadencetracker', 'actionlinkgrouptemplate', 'actionlinktemplate', 'actionplan', 'actionplanitem', 'actionplantemplate', 'actionplantemplateitem', 'actionplantemplateitemvalue', 'actionplantemplateversion', 'activescratchorg', 'activityhistory', 'activitymetric', 'additionalnumber', 'address', 'agentwork', 'agentworkskill', 'aiinsightaction', 'aiinsightfeedback', 'aiinsightreason', 'aiinsightvalue', 'airecordinsight', 'allowedemaildomain', 'announceme

In [3]:
# Creating the 'Fields' dataframe
df_fields = pd.DataFrame(columns=['ObjectName', 'FieldName', 'Type','Properties','Description'])
error_list = []

# For loop to go through the url list
for url_key in liste_of_url_key:
    url = "https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_"+url_key+".htm"
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    try :
        # Extracting Object Name
        object_name_html = soup.select('span[id="topic-title"]')
        if len(object_name_html)>0:
            object_name = object_name_html[0].text
        else : 
            object_name_html = soup.select('h1[class="helpHead1"]') 
            object_name = object_name_html[0].text

        # Extracting the fields' names 
        field_names_html = soup.select('tr > td[data-title="Field Name"]')
        if len(field_names_html) == 0 : field_names_html = soup.select('tr > td[data-title="Field"]')
        if len(field_names_html) == 0 : field_names_html = soup.select('tr > td[data-title="Name"]')
        field_names_list = []
        for i in range(len(field_names_html)):
            field_names = field_names_html[i].text
            field_names_clean = field_names.replace('\n','')
            field_names_list.append(field_names_clean)

        # Extracting the details and creating a dictionnary for each field
        field_details_html = soup.select('tr > td[data-title="Details"]')
        field_details_list = []
        for i in range(len(field_details_html)):
            field_details = field_details_html[i].text
            field_details.replace('\n','')
            if 'Type' in field_details and 'Properties'in field_details and 'Description'in field_details: 
                # Splitting and joining the elements together
                field_split = field_details.split()
                type_index = field_split.index('Type')+1
                properties_index = field_split.index('Properties')+1
                description_index = field_split.index('Description')+1
                field_type = ' '.join(field_split[type_index:properties_index-1])
                field_properties = ' '.join(field_split[properties_index:description_index-1])
                field_description = ' '.join(field_split[description_index:])
                # Creating a dictionnary with each field elements
                details_dict = {'ObjectName':object_name,'FieldName':field_names_list[i],'Type':field_type,'Properties':field_properties,'Description':field_description}
            else:
                details_dict = {'ObjectName':object_name,'FieldName':field_names_list[i],'Type':None,'Properties':None,'Description':None}
            field_details_list.append(details_dict) 

        df_fields = df_fields.append(field_details_list, ignore_index=True)
    except IndexError :
        error_list.append(url)

print(df_fields)

                 ObjectName           FieldName       Type  \
0     AcceptedEventRelation             EventId  reference   
1     AcceptedEventRelation          RelationId  reference   
2     AcceptedEventRelation       RespondedDate   dateTime   
3     AcceptedEventRelation            Response     string   
4     AcceptedEventRelation                Type     string   
...                     ...                 ...        ...   
9557          WorkTypeGroup            IsActive    boolean   
9558          WorkTypeGroup  LastReferencedDate   dateTime   
9559          WorkTypeGroup      LastViewedDate   dateTime   
9560          WorkTypeGroup                Name     string   
9561          WorkTypeGroup             OwnerId  reference   

                                             Properties  \
0                         Filter, Group, Nillable, Sort   
1                         Filter, Group, Nillable, Sort   
2                                Filter, Nillable, Sort   
3                  

In [49]:
df_fields.head(10)

,ObjectName,FieldName,Type,Properties,Description
0,AcceptedEventRelation,EventId,reference,"Filter, Group, Nillable, Sort",Indicates the ID of the event.
1,AcceptedEventRelation,RelationId,reference,"Filter, Group, Nillable, Sort",Indicates the ID of the invitee.
2,AcceptedEventRelation,RespondedDate,dateTime,"Filter, Nillable, Sort",Indicates the most recent date and time when t...
3,AcceptedEventRelation,Response,string,"Filter, Group, Nillable, Sort",Indicates the content of the response field. L...
4,AcceptedEventRelation,Type,string,"Filter, Group, Nillable, Sort","Indicates whether the invitee is a user, lead ..."
5,Account,AccountNumber,string,"Create, Filter, Group, Nillable, Sort, Update",Account number assigned to this account (not t...
6,Account,AccountSource,picklist,"Create, Filter, Group, Nillable, Sort, Update","The source of the account record. For example,..."
7,Account,AnnualRevenue,currency,"Create, Filter, Nillable, Sort, Update",Estimated annual revenue of the account.
8,Account,BillingAddress,address,"Filter, Nillable",The compound form of the billing address. Read...
9,Account,BillingCity,string,"Create, Filter, Group, Nillable, Sort, Update",Details for the billing address of this accoun...


In [4]:
print(len(error_list))

16


In [5]:
print(error_list)

['https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_article type__datacategoryselection.htm', 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_community (zone).htm', 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_consumption rate.htm', 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_consumption schedule.htm', 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_eventrelation.htm', 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_listemailindividualrecipient.htm', 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_listemailrecipientsource.htm', 'https://developer.salesforce.com/d

In [40]:
last_url_list = ['https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_articletype__datacategoryselection.htm', 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_community.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_consumption_rate.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_consumption_schedule.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_eventattendee.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_listemail_individual_recipient.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_listemail_recipient_source.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_recordvisibility.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_sessionpermsetactivations.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_topiclocalization.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_transactionsecurity.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_usermembershiprule.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_role.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_wishlist.htm','https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_wishlistitem.htm']

In [43]:
remove_objects_list = ['article type__datacategoryselection','community (zone)','consumption rate','consumption schedule','eventrelation','listemailindividualrecipient','listemailrecipientsource','recordvisibility (pilot)','sessionpermsetactivation','salesforce surveys object model','topiclocalization—beta','transactionsecuritypolicy','usermembershipsharingrule','userrole','wishlist (beta)','wishlistitem (beta)']

In [41]:
else_objects_list = ['articletype__datacategoryselection','community','consumption_rate','consumption_schedule','eventattendee','listemail_individual_recipient','listemail_recipient_source','recordvisibility','sessionpermsetactivations','topiclocalization','transactionsecurity','usermembershiprule','role','wishlist','wishlistitem']

In [45]:
for el in remove_objects_list :
    liste_of_url_key.remove(el)
for el in else_objects_list :
    liste_of_url_key.append(el)

In [31]:
first_url = 'https://developer.salesforce.com/docs/atlas.en-us.object_reference.meta/object_reference/sforce_api_objects_acceptedeventrelation.htm'
emplacement_du_driver="/Users/macbook16decamille/chromedriver"
driver = webdriver.Chrome(emplacement_du_driver)

In [32]:
# Creating the 'Fields' dataframe
error_list_2 = []

# For loop to go through the url list
for url in last_url_list:
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    try :
        # Extracting Object Name
        object_name_html = soup.select('span[id="topic-title"]')
        if len(object_name_html)>0:
            object_name = object_name_html[0].text
        else : 
            object_name_html = soup.select('h1[class="helpHead1"]') 
            object_name = object_name_html[0].text

        # Extracting the fields' names 
        field_names_html = soup.select('tr > td[data-title="Field Name"]')
        if len(field_names_html) == 0 : field_names_html = soup.select('tr > td[data-title="Field"]')
        if len(field_names_html) == 0 : field_names_html = soup.select('tr > td[data-title="Name"]')
        field_names_list = []
        for i in range(len(field_names_html)):
            field_names = field_names_html[i].text
            field_names_clean = field_names.replace('\n','')
            field_names_list.append(field_names_clean)

        # Extracting the details and creating a dictionnary for each field
        field_details_html = soup.select('tr > td[data-title="Details"]')
        field_details_list = []
        for i in range(len(field_details_html)):
            field_details = field_details_html[i].text
            field_details.replace('\n','')
            if 'Type' in field_details and 'Properties'in field_details and 'Description'in field_details: 
                # Splitting and joining the elements together
                field_split = field_details.split()
                type_index = field_split.index('Type')+1
                properties_index = field_split.index('Properties')+1
                description_index = field_split.index('Description')+1
                field_type = ' '.join(field_split[type_index:properties_index-1])
                field_properties = ' '.join(field_split[properties_index:description_index-1])
                field_description = ' '.join(field_split[description_index:])
                # Creating a dictionnary with each field elements
                details_dict = {'ObjectName':object_name,'FieldName':field_names_list[i],'Type':field_type,'Properties':field_properties,'Description':field_description}
            else:
                details_dict = {'ObjectName':object_name,'FieldName':field_names_list[i],'Type':None,'Properties':None,'Description':None}
            field_details_list.append(details_dict) 

        df_fields = df_fields.append(field_details_list, ignore_index=True)
    except IndexError :
        error_list_2.append(url)

print(df_fields)

                 ObjectName             FieldName       Type  \
0     AcceptedEventRelation               EventId  reference   
1     AcceptedEventRelation            RelationId  reference   
2     AcceptedEventRelation         RespondedDate   dateTime   
3     AcceptedEventRelation              Response     string   
4     AcceptedEventRelation                  Type     string   
...                     ...                   ...        ...   
9664        Wishlist (Beta)  WishlistProductCount        int   
9665    WishlistItem (Beta)       CurrencyIsoCode   picklist   
9666    WishlistItem (Beta)                  Name     string   
9667    WishlistItem (Beta)            Product2Id  reference   
9668    WishlistItem (Beta)            WishlistId  reference   

                                             Properties  \
0                         Filter, Group, Nillable, Sort   
1                         Filter, Group, Nillable, Sort   
2                                Filter, Nillable, Sor

In [33]:
df_fields['ObjectName'].nunique()

874

In [34]:
print(error_list_2)

[]


In [7]:
df_fields.to_csv(r'/Users/macbook16decamille/Castor/Fields.csv', index = False)

In [9]:
df_fields.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9562 entries, 0 to 9561
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ObjectName   9562 non-null   object
 1   FieldName    9562 non-null   object
 2   Type         9525 non-null   object
 3   Properties   9525 non-null   object
 4   Description  9525 non-null   object
dtypes: object(5)
memory usage: 373.6+ KB


# VERSION 46

In [25]:
## V.46
# Loading first page content
url_v46 = 'https://developer.salesforce.com/docs/atlas.en-us.220.0.api.meta/api/sforce_api_objects_acceptedeventrelation.htm'
emplacement_du_driver="/Users/macbook16decamille/chromedriver"
driver = webdriver.Chrome(emplacement_du_driver)
driver.get(url_v46)

# Accepting cookies and closing footer
time.sleep(4)
driver.find_element_by_css_selector("button#onetrust-accept-btn-handler").click()
time.sleep(2)
driver.find_element_by_css_selector("img.close-icon").click()

# Getting the list of all the pages of the current version
liste_of_url_key_v46=[]
for el in driver.find_elements_by_css_selector("span.ng-binding.ng-scope") :
    liste_of_url_key_v46.append(el.text)

# Creating the list of all the url to scrape
idx=liste_of_url_key_v46.index("AcceptedEventRelation")
idx2=liste_of_url_key_v46.index("WorkTypeGroupMember")
liste_of_url_key_v46=liste_of_url_key_v46[idx:idx2]
for idx,url in enumerate(liste_of_url_key_v46) : 
    liste_of_url_key_v46[idx]=url.lower()
print(liste_of_url_key_v46)

['acceptedeventrelation', 'account', 'accountbrand', 'accountbrandshare', 'accountcontactrelation', 'accountcleaninfo', 'accountcontactrole', 'accounthistory', 'accountinsight', 'accountownersharingrule', 'accountpartner', 'accountrelationship', 'accountrelationshiphistory', 'accountrelationshipshare', 'accountrelationshipsharerule', 'accountshare', 'accounttag', 'accountteammember', 'accountterritoryassignmentrule', 'accountterritoryassignmentruleitem', 'accountterritorysharingrule', 'accountuserterritory2view', 'actionlinkgrouptemplate', 'actionlinktemplate', 'actionplan', 'actionplanitem', 'actionplanshare', 'actionplantemplate', 'actionplantemplateitem', 'actionplantemplateitemvalue', 'actionplantemplateshare', 'actionplantemplateversion', 'activescratchorg', 'activityhistory', 'activitymetric', 'additionalnumber', 'address', 'agentwork', 'allowedemaildomain', 'announcement', 'apexclass', 'apexcomponent', 'apexlog', 'apexpage', 'apextestqueueitem', 'apextestresult', 'apextestresult

In [26]:
# Creating the 'Fields' dataframe
df_fields_v46 = pd.DataFrame(columns=['ObjectName', 'FieldName', 'Type','Properties','Description'])
error_list_v46 = []

# For loop to go through the url list
for url_key in liste_of_url_key_v46[0:10]:
    url = "https://developer.salesforce.com/docs/atlas.en-us.220.0.object_reference.meta/object_reference/sforce_api_objects_"+url_key+".htm"
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    try :
        # Extracting Object Name
        object_name_html = soup.select('span[id="topic-title"]')
        if len(object_name_html)>0:
            object_name = object_name_html[0].text
        else : 
            object_name_html = soup.select('h1[class="helpHead1"]') 
            object_name = object_name_html[0].text

        # Extracting the fields' names 
        field_names_html = soup.select('tr > td[data-title="Field Name"]')
        if len(field_names_html) == 0 : field_names_html = soup.select('tr > td[data-title="Field"]')
        if len(field_names_html) == 0 : field_names_html = soup.select('tr > td[data-title="Name"]')
        field_names_list = []
        for i in range(len(field_names_html)):
            field_names = field_names_html[i].text
            field_names_clean = field_names.replace('\n','')
            field_names_list.append(field_names_clean)

        # Extracting the details and creating a dictionnary for each field
        field_details_html = soup.select('tr > td[data-title="Details"]')
        field_details_list = []
        for i in range(len(field_details_html)):
            field_details = field_details_html[i].text
            field_details.replace('\n','')
            if 'Type' in field_details and 'Properties'in field_details and 'Description'in field_details: 
                # Splitting and joining the elements together
                field_split = field_details.split()
                type_index = field_split.index('Type')+1
                properties_index = field_split.index('Properties')+1
                description_index = field_split.index('Description')+1
                field_type = ' '.join(field_split[type_index:properties_index-1])
                field_properties = ' '.join(field_split[properties_index:description_index-1])
                field_description = ' '.join(field_split[description_index:])
                # Creating a dictionnary with each field elements
                details_dict = {'ObjectName':object_name,'FieldName':field_names_list[i],'Type':field_type,'Properties':field_properties,'Description':field_description}
            else:
                details_dict = {'ObjectName':object_name,'FieldName':field_names_list[i],'Type':None,'Properties':None,'Description':None}
            field_details_list.append(details_dict) 

        df_fields_v46 = df_fields_v46.append(field_details_list, ignore_index=True)
    except IndexError :
        error_list_v46.append(url)

print(df_fields_v46)

                  ObjectName                FieldName       Type  \
0      AcceptedEventRelation                  EventId  reference   
1      AcceptedEventRelation               RelationId  reference   
2      AcceptedEventRelation            RespondedDate   dateTime   
3      AcceptedEventRelation                 Response     string   
4      AcceptedEventRelation                     Type     string   
..                       ...                      ...        ...   
267  AccountOwnerSharingRule           DeveloperName      string   
268  AccountOwnerSharingRule                  GroupId  reference   
269  AccountOwnerSharingRule   OpportunityAccessLevel   picklist   
270  AccountOwnerSharingRule                     Name     string   
271  AccountOwnerSharingRule            UserOrGroupId  reference   

                                            Properties  \
0                        Filter, Group, Nillable, Sort   
1                        Filter, Group, Nillable, Sort   
2        

In [27]:
print(len(error_list_v46))

0
